# Cafe Dataset Cleanup


## Library Setup

For this data cleaning process we'll be using pandas to handle our dataset, numpy for vectorised operations and random for categorised imputations.

In [1]:
import pandas as pd
import numpy as np
import random

## Load Data

First, we load our dataset into a pandas dataframe object and initilise our dictionaries

In [2]:
path = "/workspaces/Data-Science-Projects/EDA/Cafe/data/dirty_cafe_sales.csv"
df = pd.read_csv(path)

df.shape

(10000, 8)

In [3]:
menu = {
    "Cookie": 1.0,
    "Tea": 1.5,
    "Coffee": 2.0,
    "Cake": 3.0,
    "Juice": 3.0,
    "Sandwich": 4.0,
    "Smoothie": 4.0,
    "Salad": 5.0,
}

price_item_dict = {}
for item, price in menu.items():
    price_item_dict.setdefault(price, []).append(item)

price_item_dict


{1.0: ['Cookie'],
 1.5: ['Tea'],
 2.0: ['Coffee'],
 3.0: ['Cake', 'Juice'],
 4.0: ['Sandwich', 'Smoothie'],
 5.0: ['Salad']}

In [4]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [5]:
df.tail()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02
9999,TXN_6170729,Sandwich,3,4.0,12.0,Cash,In-store,2023-11-07


In [6]:
df.dtypes

Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object

In [7]:
df.isna().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

From a preliminary dataset visualisation and analysis, our dataset contains various data incosistenciese, missing values and incorrect data types for numerical and date data. Our data has 10 000 records and 8 columns.

## Data Cleaning

To begin our data cleaning pipeline, we'll rename the columns to increase efficiency while coding.

In [8]:
df.columns = ['Txn_ID', 'Item', 'Quantity', 'Unit_Price', 'Total', 'Payment', 'Location', 'Date']
df.head(0)

,Txn_ID,Item,Quantity,Unit_Price,Total,Payment,Location,Date


### Data Type Conversion

There are invalid values that we'll convert to null values and columns that we will convert to appropriate data types

In [9]:
df.duplicated("Txn_ID").sum()

np.int64(0)

Our unique identifier is not duplicated, letting us know that every record in the dataset is an individual transaction.

In [10]:
df.head()

,Txn_ID,Item,Quantity,Unit_Price,Total,Payment,Location,Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [11]:
df.loc[df.Item.isin(["UNKNOWN", "ERROR"]), "Item"] = pd.NA
df.Quantity = pd.to_numeric(df.Quantity, errors="coerce").astype("Int64")
df.Unit_Price = pd.to_numeric(df.Unit_Price, errors="coerce").astype("Float64")
df.Total = pd.to_numeric(df.Total, errors="coerce").astype("Float64")
df.loc[df.Location.isin(["UNKNOWN", "ERROR"]), "Location"] = pd.NA
df.Date = pd.to_datetime(df.Date, errors="coerce")

df.dtypes

Txn_ID                object
Item                  object
Quantity               Int64
Unit_Price           Float64
Total                Float64
Payment               object
Location              object
Date          datetime64[ns]
dtype: object

## Data Pruning

With our data types corrected, we'll begin removing records that don't have sufficient data for data imputation or data recovery. We'll get rid of records where the mathematical relationship `Total = Quantity * Unit_Price` cannot be utilised to recovery financial data, data with missing date values, and data that cannot be recovered by mapping to our dictionaries.

In [12]:
df.shape

(10000, 8)

If Item, Quantity and Unit_Price columns are empty (null).

In [13]:
mask = df.Item.isna() & df.Quantity.isna() & df.Unit_Price.isna()
df = df.drop(df[mask].index)
df.shape

(9997, 8)

If Quantity, Unit_Price and Total columns are empty (null).

In [14]:
mask = df.Quantity.isna() & df.Unit_Price.isna() & df.Total.isna()
df = df.drop(df[mask].index)
df.shape

(9997, 8)

If Item, Unit_Price and Total columns are null.

In [15]:
mask = df.Item.isna() & df.Unit_Price.isna() & df.Total.isna()
df = df.drop(df[mask].index)
df.shape

(9994, 8)

If Item, Quantity and Total columns are null.

In [16]:
mask = df.Item.isna() & df.Quantity.isna() & df.Total.isna()
df = df.drop(df[mask].index)
df.shape

(9994, 8)

If Quantity and Total columns are null.

In [17]:
mask = df.Quantity.isna() & df.Total.isna()
df = df.drop(df[mask].index)
df.shape

(9974, 8)

Drop null date records.

In [18]:
df = df.dropna(subset=["Date"])
df.shape

(9514, 8)

## Data Imputation

### Function Definition

If Item is present, Unit_Price is empty and Item is in the menu dictionary, acquire the Unit_Price from dictionary.

In [19]:
def impute_unit_price(dataframe, menu_dict):
    dataframe = dataframe.copy() 
    
    mask = (
        dataframe['Item'].notna() & 
        dataframe['Unit_Price'].isna() & 
        dataframe['Item'].isin(menu_dict.keys())
    )
    
    dataframe.loc[mask, 'Unit_Price'] = dataframe.loc[mask, 'Item'].map(menu_dict)
    
    return dataframe

If Unit_Price is present, Item is empty and Unit_Price is present in the price_item_dict dictionary, acquire the Item from the dictionary. In cases where there is more than one option for Item, randomly select an Item.

In [20]:
def impute_item(dataframe, price_item_dict):
    dataframe = dataframe.copy()  
    
    mask = (
        dataframe['Unit_Price'].notna() & 
        dataframe['Item'].isna() & 
        dataframe['Unit_Price'].isin(price_item_dict.keys())
    )
    
    def select_random_item(price):
        return random.choice(price_item_dict[price])
    
    dataframe.loc[mask, 'Item'] = dataframe.loc[mask, 'Unit_Price'].map(select_random_item)
    
    return dataframe

Use the mathematical relationship `Total = Quantity × Unit_Price` where the is sufficient data to calculate the missing data

In [21]:
def calculate_missing_values(dataframe):
    dataframe = dataframe.copy() 
    
    dataframe.Quantity = np.where(
        dataframe.Unit_Price.notna() & dataframe.Total.notna(), 
        dataframe.Total / dataframe.Unit_Price, 
        dataframe.Quantity
    )
    
    dataframe.Unit_Price = np.where(
        dataframe.Quantity.notna() & dataframe.Total.notna(), 
        dataframe.Total / dataframe.Quantity, 
        dataframe.Unit_Price
    )
    
    dataframe.Total = np.where(
        dataframe.Quantity.notna() & dataframe.Unit_Price.notna(), 
        dataframe.Quantity * dataframe.Unit_Price, 
        dataframe.Total
    )
    
    return dataframe

### Data Iputation

#### Item, Quantity, Unit_Price and Total
Check missing values before imputation

In [22]:
print(f"Missing Unit Price with Item: {df[df.Item.notna() & df.Unit_Price.isna()].shape[0]}")
print(f"Missing Item with Unit Price: {df[df.Item.isna() & df.Unit_Price.notna()].shape[0]}")
print(f"Missing Quatinties: {df.Quantity.isna().sum()}")
print(f"Missing Unit Prices: {df.Unit_Price.isna().sum()}")
print(f"Missing Totals: {df.Total.isna().sum()}")


Missing Unit Price with Item: 455
Missing Item with Unit Price: 876
Missing Quatinties: 431
Missing Unit Prices: 500
Missing Totals: 453


Use the functions defined to impute Unit_Price and Item columns, and calculate the financial data.

In [23]:
df = impute_unit_price(dataframe=df, menu_dict=menu)
df = calculate_missing_values(dataframe=df)
df = impute_item(dataframe=df, price_item_dict=price_item_dict)

Check missing values after imputation

In [24]:
df[df[["Item", "Quantity", "Unit_Price", "Total"]].isna().any(axis=1)]

,Txn_ID,Item,Quantity,Unit_Price,Total,Payment,Location,Date


#### Location and Payment
Check missing values before imputation

In [25]:
print(f"Missing Location: {df.Location.isna().sum()}")
print(f"Missing Payment: {df.Payment.isna().sum()}")
print(f"Total: {df[(df.Location.isna()) | (df.Payment.isna())].shape[0]}") # covers instances where both are missing as well

# df_lo_pay = df[(df.Location.isna()) | (df.Payment.isna())]
# df_lo_pay.shape

Missing Location: 3770
Missing Payment: 2430
Total: 5229


Define a function to determine the mode of grouped Items

In [26]:
mode = lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x

Use function to impute modal values into Location and Payment columns.

In [27]:
df.Payment = df.groupby('Item').Payment.transform(mode)
df.Location = df.groupby('Item').Location.transform(mode)

Check missing values after imputation

In [28]:
df[df[["Location", "Payment"]].isna().any(axis=1)]

,Txn_ID,Item,Quantity,Unit_Price,Total,Payment,Location,Date


Check the entire dataframe for missing values

In [29]:
df.isna().sum()

Txn_ID        0
Item          0
Quantity      0
Unit_Price    0
Total         0
Payment       0
Location      0
Date          0
dtype: int64

In [30]:
df.shape

(9514, 8)

After our data cleaning pipeline, our dataset has 9514 records with no missing or invalid data.